In [1]:
from src.neo4jdataloader import DataLoader
from src.neo4jcrud import UrbsNeo4JDatabase

In [2]:
NEO4J_URI = 'bolt://172.17.0.2:7687'
NEO4J_USER = 'neo4j'
NEO4J_PASSWORD = 'neo4j2018'

conn = UrbsNeo4JDatabase(NEO4J_URI, NEO4J_USER, NEO4J_PASSWORD)

In [3]:
conn.delete_all()

In [4]:
dataloader = DataLoader()

In [5]:
trechositinerarios_source_path = '/home/altieris/datascience/data/curitibaurbs/processed/trechositinerarios/'
trechositinerarios = dataloader.load_data(trechositinerarios_source_path)
dataloader.create_empresas_onibus(trechositinerarios, conn)
dataloader.create_categorias_onibus(trechositinerarios, conn)

In [6]:
pontos_linha_source_path = '/home/altieris/datascience/data/curitibaurbs/processed/pontoslinha/'
pontos_linha = dataloader.load_data(pontos_linha_source_path)
dataloader.create_bus_stops(pontos_linha,conn)

In [7]:
dataloader.create_routes(conn)

In [9]:
veiculos = dataloader.load_data('/home/altieris/datascience/data/curitibaurbs/processed/veiculos/')

In [10]:
dataloader.create_vehicles(veiculos,conn)

In [11]:
veiculos_df = veiculos.select('veic','cod_linha').filter("year ='2019' and month='03' and day = '14' ").distinct().toPandas()

In [ ]:
for index, row in veiculos_df.iterrows():
    dataloader.create_positions(veiculos,row['veic'],row['cod_linha'],conn) 

In [7]:
conn.close()

In [ ]:
# MATCH (p:Position) 
# WITH p ORDER BY p.dt_event DESC
# WITH collect(p) as entries 
# FOREACH(i in RANGE(0, length(entries)-2) | 
#   FOREACH(e1 in [entries[i]] | 
#     FOREACH(e2 in [entries[i+1]] |  
#        MERGE (e2)-[:MOVES_TO { line_code: $line_code }]->(e1))))